
 <img src="metoo.jpeg" align=center>
 <center>
<font>
<h1> Twitter Analysis: #MeToo Movement </h1>
<h3>Priyanka Khanal </h3>
<h3>March 12, 2019</h3>
</font>
    </center>

<center> <h2> Importing Various Python Libraries </h2> </center>

In [5]:
from twitter_keys import consumer_key, consumer_secret, access_token, access_secret
import tweepy #for twitter API
import pickle
import pandas as pd # To Handle Data
import numpy as np #for number computing
from collections import Counter
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from textblob import TextBlob 
from wordcloud import WordCloud, STOPWORDS 
import re
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\s524702\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


<center> <h2> Authentication </h2> </center>

In [4]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)
MeToo_tweets=[]
for tweet in tweepy.Cursor(api.search, q='#MeToo',lang='en', tweet_mode='extended').items(10000):
    MeToo_tweets.append(tweet)
#with open("MeToo.pkl", 'wb') as f:
    #pickle.dump(MeToo_tweets, f)
with open('MeToo.pkl', 'rb') as f:
    MeToo_tweets = pickle.load(f)

<center><h2> Printing the tweets in a form of a table </h2> </center>

In [24]:
data = pd.DataFrame(data=[tweet.full_text for tweet in MeToo_tweets], columns=['MeTooTweets'])
data['MeToolen']  = np.array([len(tweet.full_text) for tweet in MeToo_tweets])
data['MeTooDate'] = np.array([tweet.created_at for tweet in MeToo_tweets])
data['MeTooSource'] = np.array([tweet.source for tweet in MeToo_tweets])
data['MeTooLikes']  = np.array([tweet.favorite_count for tweet in MeToo_tweets])
data['MeTooRTs']    = np.array([tweet.retweet_count for tweet in MeToo_tweets])
display(data.head(20))

,MeTooTweets,MeToolen,MeTooDate,MeTooSource,MeTooLikes,MeTooRTs
0,RT @paradiseflossed: Vast majority of #MeToo s...,140,2019-03-12 14:30:10,Twitter Web App,0,7
1,"Sarah McLachlan on music, the Junos and the im...",118,2019-03-12 14:29:46,Twitter Web Client,0,0
2,@MuslimIQ @gidgetdoggo Support a Religion that...,166,2019-03-12 14:29:44,Twitter for iPhone,0,0
3,@coco_miyaki I see those fans didn't learn any...,78,2019-03-12 14:29:38,Twitter for Android,0,0
4,RT @michellemalkin: If you are wondering what ...,144,2019-03-12 14:29:33,Twitter for iPhone,0,6440
5,Why do we have to celebrate International Wome...,267,2019-03-12 14:29:19,The Social Jukebox,2,0
6,RT @Red_Pastures: I remember the trite reply h...,140,2019-03-12 14:29:06,Twitter for Android,0,4
7,Amber Heard used the #MeToo movement without a...,123,2019-03-12 14:29:03,Twitter for Android,1,0
8,@bucinjun wdym choi innocent he has #metoo sca...,87,2019-03-12 14:29:03,Twitter for iPhone,0,0
9,"$10,000. Unrestricted Grant Support. Triangle-...",279,2019-03-12 14:29:00,LinkedIn,0,0


<center><h2> Lexical Diversity</h2></center>

In [7]:
MeToolexemes = []
for tweet in MeToo_tweets:
    MeToolexemes.extend(tweet.full_text.split())
def lexical_diversity(tokens):
    return len(set(tokens))/len(tokens)
MeToold=lexical_diversity(MeToolexemes)

<center><h2>Cleaning Data and Sentiment Analysis</h2></center>

In [8]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt  

In [9]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 1
    elif analysis.sentiment.polarity == 0:
        return 0
    else:
        return -1

In [10]:
data['MeTooClean'] = np.vectorize(remove_pattern)(data['MeTooTweets'], "@[\w]*")
data['MeTooClean'] = data['MeTooClean'].str.replace("[^a-zA-Z#]", " ")
data['MeTooClean']= data['MeTooClean'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>4]))
data['SA'] = np.array([analyze_sentiment(tweet) for tweet in data['MeTooTweets'] ])
data.head()

,MeTooTweets,MeToolen,MeTooDate,MeTooSource,MeTooLikes,MeTooRTs,MeTooClean,SA
0,RT @paradiseflossed: Vast majority of #MeToo s...,140,2019-03-12 14:30:10,Twitter Web App,0,7,majority #MeToo stories legit #AmberHeard soci...,0
1,"Sarah McLachlan on music, the Junos and the im...",118,2019-03-12 14:29:46,Twitter Web Client,0,0,Sarah McLachlan music Junos impact Lilith #MeT...,1
2,@MuslimIQ @gidgetdoggo Support a Religion that...,166,2019-03-12 14:29:44,Twitter for iPhone,0,0,Support Religion beheads women because sheet s...,0
3,@coco_miyaki I see those fans didn't learn any...,78,2019-03-12 14:29:38,Twitter for Android,0,0,those learn anything #metoo movement,0
4,RT @michellemalkin: If you are wondering what ...,144,2019-03-12 14:29:33,Twitter for iPhone,0,6440,wondering Media Matters Media Twitter trying d...,0


In [13]:
pos_tweets = [ tweet for index, tweet in enumerate(data['MeTooClean']) if data['SA'][index] > 0]
neu_tweets = [ tweet for index, tweet in enumerate(data['MeTooClean']) if data['SA'][index] == 0]
neg_tweets = [ tweet for index, tweet in enumerate(data['MeTooClean']) if data['SA'][index] < 0]

In [14]:
pos=format(len(pos_tweets)*100/len(data['MeTooClean']))
neu=format(len(neu_tweets)*100/len(data['MeTooClean']))
neg=format(len(neg_tweets)*100/len(data['MeTooClean']))
percent=(pos,neu,neg)
lab=["Positive tweets","Neutral Tweets","Negative Tweets"]
plt.pie(percent, labels=lab, startangle=90, autopct='%.1f%%')
plt.title('Sentiment Analysis of #MeToo Movement Tweets')
plt.show()


<img src = "sent_any.png">

<center><h2> WordCloud </h2> </center>

In [15]:
#wordCloud for #meToo
words= data['MeTooClean']
stopwords = set(STOPWORDS)
stopwords.update(["gkhowe29", "amp", "metoo", "https", "V9GHLyRRXQ","co","gi","RT","Amber","michellemalkin","omekongo","Candace","coco_miyaki","jogncardillo","bucinjun","TuckerCarlson","MeTooTweets","MeTooRTs","GidgetDoggo","im","choi","isn","triangle","Twitter","dwQZEKSUv","dtype"])
wordcloud = WordCloud(collocations=False,max_words=100,stopwords=stopwords, background_color='white',max_font_size=40, scale=3).generate(str(words))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

<img src="wordcloud_metoo.png">

In [26]:
mostReTweets=data.sort_values(by='MeTooRTs', ascending=False)
mostReTweets.head(10)

,MeTooTweets,MeToolen,MeTooDate,MeTooSource,MeTooLikes,MeTooRTs
5596,RT @MiamiHerald: Long before #MeToo became the...,140,2019-03-12 00:27:48,Twitter for iPhone,0,18342
7540,RT @MiamiHerald: Long before #MeToo became the...,140,2019-03-11 21:45:18,Twitter for Android,0,18342
5771,RT @Alyssa_Milano: Don't let this performance ...,139,2019-03-12 00:11:52,Twitter for Android,0,17079
616,RT @Alyssa_Milano: Don't let this performance ...,139,2019-03-12 13:24:12,Twitter for iPhone,0,17079
4026,"RT @DanielleMuscato: Ladies, a question for yo...",140,2019-03-12 02:56:01,Twitter for Android,0,11074
8258,RT @RealCandaceO: The media has gone NUTS over...,140,2019-03-11 21:00:04,Twitter for Android,0,9898
6306,RT @RealCandaceO: The media has gone NUTS over...,140,2019-03-11 23:24:42,Twitter for iPad,0,9897
3932,RT @RealCandaceO: The media has gone NUTS over...,140,2019-03-12 03:05:39,Twitter Web Client,0,9895
4709,RT @RealCandaceO: The media has gone NUTS over...,140,2019-03-12 01:48:00,Twitter Web App,0,9895
3448,RT @RealCandaceO: The media has gone NUTS over...,140,2019-03-12 04:07:57,Twitter for iPad,0,9895


<center><h2> Tarana Burke: Tweets Analysis </h2> </center>

In [17]:
TBurke_tweets = api.user_timeline('TaranaBurke', count=40)
#with open('TBurke_tweets.pkl', 'wb') as f:
    #pickle.dump(TBurke_tweets, f)
with open('TBurke_tweets.pkl', 'rb') as f:
    TBurke_tweets = pickle.load(f)
data = pd.DataFrame(data=[tweet.text for tweet in TBurke_tweets], columns=['TBurkeTweets'])
data['TBlen']  = np.array([len(tweet.text) for tweet in TBurke_tweets])
data['TBDate'] = np.array([tweet.created_at for tweet in TBurke_tweets])
data['TBSource'] = np.array([tweet.source for tweet in TBurke_tweets])
data['TBLikes']  = np.array([tweet.favorite_count for tweet in TBurke_tweets])
data['TBRTs']    = np.array([tweet.retweet_count for tweet in TBurke_tweets])
display(data.head(10))

,TBurkeTweets,TBlen,TBDate,TBSource,TBLikes,TBRTs
0,@ischafer https://t.co/sBmFfJ1zeu,33,2019-03-12 17:27:29,Twitter for iPhone,9,4
1,@SarahKSilverman The SQUEEZE BOTTLE!! https://...,61,2019-03-12 15:08:58,Twitter for iPhone,35,3
2,@PDXtroyboy @cmclymer https://t.co/39tWNT9bcV,45,2019-03-12 15:07:24,Twitter for iPhone,38,5
3,@tkalogeropoulos Anything else is blasphemy!,44,2019-03-12 15:06:42,Twitter for iPhone,6,3
4,@quiqonky https://t.co/9xn1R200ck,33,2019-03-12 15:06:12,Twitter for iPhone,27,4
5,@VictoriaCliett1 https://t.co/u3eYneBWxj,40,2019-03-12 15:05:44,Twitter for iPhone,12,3
6,Random: I want to know who decided grape jelly...,140,2019-03-12 14:21:01,Twitter for iPhone,3903,218
7,@imaniperry @DrCamikaRoyal @michelealexandr Th...,85,2019-03-12 14:10:21,Twitter for iPhone,2,3
8,@imaniperry @michelealexandr OMG!!!! 😍😍😍😍Tunda...,71,2019-03-12 03:48:27,Twitter for iPhone,11,4
9,Lessgo! https://t.co/jNBImEE9hn,31,2019-03-12 00:02:25,Twitter for iPhone,1109,184


<center><h2> Lexical Diversity </h2> </center>

In [18]:
TBlexemes = []
for tweet in TBurke_tweets:
    TBlexemes.extend(tweet.text.split())
def lexical_diversity(tokens):
    return len(set(tokens))/len(tokens)
TBld=lexical_diversity(TBlexemes)
print(TBld)

0.7401247401247402


<center> <h2> Cleaning of Tweets </h2> </center> 

In [19]:
data['TBurkeClean'] = np.vectorize(remove_pattern)(data['TBurkeTweets'], "@[\w]*")
data['TBurkeClean'] = data['TBurkeClean'].str.replace("[^a-zA-Z#]", " ")
data['TBurkeClean']= data['TBurkeClean'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>4]))
data['TBurkeSA'] = np.array([analyze_sentiment(tweet) for tweet in data['TBurkeTweets'] ])
data.head()

,TBurkeTweets,TBlen,TBDate,TBSource,TBLikes,TBRTs,TBurkeClean,TBurkeSA
0,@ischafer https://t.co/sBmFfJ1zeu,33,2019-03-12 17:27:29,Twitter for iPhone,9,4,https sBmFfJ,0
1,@SarahKSilverman The SQUEEZE BOTTLE!! https://...,61,2019-03-12 15:08:58,Twitter for iPhone,35,3,SQUEEZE BOTTLE https APVzLibJLX,0
2,@PDXtroyboy @cmclymer https://t.co/39tWNT9bcV,45,2019-03-12 15:07:24,Twitter for iPhone,38,5,https,0
3,@tkalogeropoulos Anything else is blasphemy!,44,2019-03-12 15:06:42,Twitter for iPhone,6,3,Anything blasphemy,0
4,@quiqonky https://t.co/9xn1R200ck,33,2019-03-12 15:06:12,Twitter for iPhone,27,4,https,0


<center><h2> Word Cloud </h2></center>

In [21]:
words= data['TBurkeClean']
stopwords = set(STOPWORDS)
stopwords.update(["https","jNBImEE","sBmFfJ","eYneBWxj","APVzLibJLX","fZBFw","dPAjmYGt","RSDeMftg","jRXqFdFnc"])
wordcloud = WordCloud(collocations=False,max_words=40,stopwords=stopwords, background_color='white',max_font_size=40, scale=3).generate(str(words))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

<img src="TBurke_tweets.png">